In [9]:
# Import appropriate modules
import pandas as pd
import datetime
import numpy as np
import yfinance as yf
from pathlib import Path
from datetime import datetime
import hvplot
import hvplot.pandas

In [10]:
def retrieve_yahoo_data(ticker = 'spy', start_date = '2007-07-01', end_date = '2021-10-03'):
    try:
        # get data based on ticker
        yahoo_data = yf.Ticker(ticker)
        print(f"Processing Ticker {ticker}")
        # select data using start date and end data and calculate the daily return
        price_df = yahoo_data.history(start=start_date, end=end_date).Close
        price_df.name = ticker
        # if no data retrieved raise exception
        if price_df.shape[0] == 0:
            raise Exception("No Prices.")
        return price_df
    # handle exception
    except Exception as ex:
        print(f"Sorry, Data not available for '{ticker}': Exception is {ex}")

In [65]:
# Set up the variable related to tickers and tickers data
ticker_data_dict = {}
ticker_list = ["^VIX", "spy", "DX-Y.NYB", "tlt", "ief", "gld", "slv", "CL=F", "USDJPY=X", "VXX"]

# Retrieve Tickers
def retrieve_tickers(ticker_data_dict, ticker_list):
    for ticker in ticker_list:
        ticker_data = retrieve_yahoo_data(ticker)
        ticker_data_dict[ticker] = ticker_data
        
    return ticker_data_dict
        
ticker_data_dict = retrieve_tickers(ticker_data_dict, ticker_list)
ticker_data_df = pd.DataFrame(ticker_data_dict)
# ticker_data_df = ticker_data_df.dropna()
# Display the ticker data related data frame
ticker_data_df.head()

Processing Ticker ^VIX
Processing Ticker spy
Processing Ticker DX-Y.NYB
Processing Ticker tlt
Processing Ticker ief
Processing Ticker gld
Processing Ticker slv
Processing Ticker CL=F
Processing Ticker USDJPY=X
Processing Ticker VXX


,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX
Date,,,,,,,,,,
2007-07-02,15.40,114.157913,81.389999,56.166187,58.142723,65.019997,12.588,71.089996,122.230003,NaN
2007-07-03,14.92,114.571518,NaN,55.823769,57.970230,64.739998,12.532,NaN,122.470001,NaN
2007-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.660004,NaN
2007-07-05,15.48,114.451256,81.540001,55.198105,57.646790,64.419998,12.465,71.809998,122.949997,NaN
2007-07-06,14.72,115.052895,81.459999,54.974232,57.510231,64.959999,12.680,72.809998,123.360001,NaN


In [58]:
returns_df=ticker_data_df.pct_change()
returns_all_but_VIX=returns_df.drop(columns=['^VIX'])

In [59]:
cum_returns=(1+returns_df).cumprod()
cum_returns_all_but_vix=(1+returns_all_but_VIX).cumprod()
cum_returns_all_but_vix

,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX
Date,,,,,,,,,
2007-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-07-03,1.003623,1.000000,0.993903,0.997033,0.995694,0.995551,1.000000,1.001963,NaN
2007-07-04,1.003623,1.000000,0.993903,0.997033,0.995694,0.995551,1.000000,1.003518,NaN
2007-07-05,1.002569,1.001843,0.982765,0.991471,0.990772,0.990229,1.010128,1.005890,NaN
2007-07-06,1.007840,1.000860,0.978778,0.989122,0.999077,1.007309,1.024195,1.009245,NaN
...,...,...,...,...,...,...,...,...,...
2021-09-27,3.877437,1.147315,2.606017,1.984771,2.516764,1.663489,1.061331,0.906193,0.224331
2021-09-28,3.799300,1.152107,2.565423,1.977204,2.492310,1.650779,1.059080,0.907829,0.247740
2021-09-29,3.805694,1.159110,2.569874,1.979268,2.481083,1.584843,1.052609,0.912444,0.251356


In [60]:
# Are levels correlated? -- YES
ticker_data_df.corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX
^VIX,1.000000,-0.262252,-0.177535,-0.153039,-0.238696,-0.140994,-0.116533,-0.154325,-0.236938,0.375032
spy,-0.262252,1.000000,0.694436,0.885131,0.866806,0.481797,-0.113056,-0.482599,0.529329,-0.868338
DX-Y.NYB,-0.177535,0.694436,1.000000,0.770690,0.754411,0.123592,-0.455048,-0.830481,0.676866,0.274011
tlt,-0.153039,0.885131,0.770690,1.000000,0.982691,0.623438,-0.040165,-0.623991,0.398247,-0.356791
ief,-0.238696,0.866806,0.754411,0.982691,1.000000,0.680951,0.045964,-0.569720,0.344841,-0.469303
gld,-0.140994,0.481797,0.123592,0.623438,0.680951,1.000000,0.714587,-0.023258,-0.360724,-0.539503
slv,-0.116533,-0.113056,-0.455048,-0.040165,0.045964,0.714587,1.000000,0.477390,-0.711848,-0.681930
CL=F,-0.154325,-0.482599,-0.830481,-0.623991,-0.569720,-0.023258,0.477390,1.000000,-0.514512,-0.340802
USDJPY=X,-0.236938,0.529329,0.676866,0.398247,0.344841,-0.360724,-0.711848,-0.514512,1.000000,0.049850
VXX,0.375032,-0.868338,0.274011,-0.356791,-0.469303,-0.539503,-0.681930,-0.340802,0.049850,1.000000


In [61]:
#Are returns correlated? YES, the most
returns_df.corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX
^VIX,1.000000,-0.718837,0.093661,0.334242,0.344377,-0.010204,-0.164473,-0.130616,-0.110360,0.886326
spy,-0.718837,1.000000,-0.182415,-0.432243,-0.430473,0.031160,0.220354,0.173950,0.212774,-0.773624
DX-Y.NYB,0.093661,-0.182415,1.000000,0.011827,-0.053894,-0.406660,-0.418170,-0.101696,0.051795,0.072226
tlt,0.334242,-0.432243,0.011827,1.000000,0.911058,0.153195,-0.007272,-0.120446,-0.128952,0.280800
ief,0.344377,-0.430473,-0.053894,0.911058,1.000000,0.192802,0.026647,-0.105590,-0.190336,0.312000
gld,-0.010204,0.031160,-0.406660,0.153195,0.192802,1.000000,0.794172,0.071690,-0.037653,-0.066436
slv,-0.164473,0.220354,-0.418170,-0.007272,0.026647,0.794172,1.000000,0.124697,0.002264,-0.227686
CL=F,-0.130616,0.173950,-0.101696,-0.120446,-0.105590,0.071690,0.124697,1.000000,0.035922,-0.163233
USDJPY=X,-0.110360,0.212774,0.051795,-0.128952,-0.190336,-0.037653,0.002264,0.035922,1.000000,-0.013628
VXX,0.886326,-0.773624,0.072226,0.280800,0.312000,-0.066436,-0.227686,-0.163233,-0.013628,1.000000


In [62]:
# Is the VIX level correlted with returns? -- Not do much. It's much more correlated with levels,
# or vIX returns correlated with returns

vix_level_and_price_returns_df=pd.concat([ticker_data_df['^VIX'],returns_all_but_VIX], axis=1)
vix_level_and_price_returns_df.corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX
^VIX,1.000000,-0.137021,0.037840,0.071523,0.080746,0.017460,-0.024428,-0.075762,-0.054239,0.223272
spy,-0.137021,1.000000,-0.182415,-0.432243,-0.430473,0.031160,0.220354,0.173950,0.212774,-0.773624
DX-Y.NYB,0.037840,-0.182415,1.000000,0.011827,-0.053894,-0.406660,-0.418170,-0.101696,0.051795,0.072226
tlt,0.071523,-0.432243,0.011827,1.000000,0.911058,0.153195,-0.007272,-0.120446,-0.128952,0.280800
ief,0.080746,-0.430473,-0.053894,0.911058,1.000000,0.192802,0.026647,-0.105590,-0.190336,0.312000
gld,0.017460,0.031160,-0.406660,0.153195,0.192802,1.000000,0.794172,0.071690,-0.037653,-0.066436
slv,-0.024428,0.220354,-0.418170,-0.007272,0.026647,0.794172,1.000000,0.124697,0.002264,-0.227686
CL=F,-0.075762,0.173950,-0.101696,-0.120446,-0.105590,0.071690,0.124697,1.000000,0.035922,-0.163233
USDJPY=X,-0.054239,0.212774,0.051795,-0.128952,-0.190336,-0.037653,0.002264,0.035922,1.000000,-0.013628
VXX,0.223272,-0.773624,0.072226,0.280800,0.312000,-0.066436,-0.227686,-0.163233,-0.013628,1.000000


In [ ]:
vix_level_and_cum_price_returns_df=pd.concat([ticker_data_df['^VIX'],cum_returns_all_but_vix], axis=1)
vix_level_and_cum_price_returns_df.hvplot()

In [ ]:
vix_level_adjusted=ticker_data_df['^VIX']/10
vix_level_adjusted.hvplot()

In [ ]:
vix_level_adj_and_cum_price_returns_df=pd.concat([vix_level_adjusted,cum_returns_all_but_vix], axis=1)
vix_level_adj_and_cum_price_returns_df.hvplot()

In [ ]:
ticker_data_df[["^VIX", "VXX"]].dropna().hvplot()

In [76]:
VIX_and_VXX_for_plot=pd.concat([ticker_data_df["^VIX"], ticker_data_df["VXX"]/5],axis=1).dropna()
VIX_and_VXX_for_plot.hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [81]:
VIX_and_VXX_for_plot=pd.concat([ticker_data_df["^VIX"], ticker_data_df["VXX"]],axis=1).dropna()
returns_vix_and_vxx=VIX_and_VXX_for_plot.pct_change().dropna()
returns_vix_and_vxx.head()

,^VIX,VXX
Date,,
2018-01-29,0.195164,0.069414
2018-01-30,0.068642,0.032792
2018-01-31,-0.084517,0.003273
2018-02-01,-0.005170,-0.050343
2018-02-02,0.285078,0.128594


In [87]:
# Cases of interest
# When both go up together
vix_and_vxx_positive_df=returns_vix_and_vxx[(returns_vix_and_vxx["^VIX"]*returns_vix_and_vxx['VXX']>0) & 
                     (returns_vix_and_vxx["^VIX"]>0)]

In [137]:
total_number_of_ups_together=vix_and_vxx_positive_df.describe().iloc[0,0]
vix_and_vxx_positive_df.describe()

,^VIX,VXX
count,321.000000,321.000000
mean,0.094033,0.047187
std,0.107635,0.052817
min,0.000800,0.000719
25%,0.034091,0.014182
50%,0.063636,0.030641
75%,0.112665,0.059621
max,1.155979,0.370602


In [92]:
vix_and_vxx_positive_df.hvplot(kind='box')

:BoxWhisker   [Variable]   (value)

In [174]:
# It is 3% a good threshold? It would if most of the time vxx is also positive when vix is above threshold
results=[]

for threshold_bps in range(150,1000,5):
    threshold=threshold_bps/10000
    returns_vxx_negative_with_vix_above_threshold=returns_vix_and_vxx[(returns_vix_and_vxx["^VIX"]>threshold) &
                                                                  (returns_vix_and_vxx["VXX"]<0)]

                                                                  
    cases_vxx_negative_and_vix_above_threshold=returns_vxx_negative_with_vix_above_threshold.describe().iloc[0,0]
    cases_vix_above_threshold=returns_vix_and_vxx[returns_vix_and_vxx["^VIX"]>threshold].describe().iloc[0,0]
    
    results.append({'Threshold return (%)':threshold*100, 
                     "cases_vxx_negative_and_vix_above_threshold":cases_vxx_negative_and_vix_above_threshold,
                     "cases_vix_above_threshold":cases_vix_above_threshold,
                     "Proportion of fail cases (%)": cases_vxx_negative_and_vix_above_threshold/cases_vix_above_threshold*100
                    })
    
results_df=pd.DataFrame(results)

results_df.head()

,Threshold return (%),cases_vxx_negative_and_vix_above_threshold,cases_vix_above_threshold,Proportion of fail cases (%)
0,1.50,38.0,334.0,11.377246
1,1.55,38.0,333.0,11.411411
2,1.60,37.0,330.0,11.212121
3,1.65,36.0,325.0,11.076923
4,1.70,36.0,323.0,11.145511


In [169]:
curve=results_df.hvplot(x='Threshold return (%)', 
                  y='Proportion of fail cases (%)',
                  title='Determination of Threshold for VIX return prediction'
                 )
amount_of_cases=results_df[(results_df['Threshold return (%)']>2) &
                           (results_df['Threshold return (%)']<4)].hvplot.bar(x='Threshold return (%)', 
                  y=['cases_vxx_negative_and_vix_above_threshold','cases_vix_above_threshold'],
                  title='Determination of Threshold for VIX return prediction'
                 )
curve+amount_of_cases

:Layout
   .Curve.I :Curve   [Threshold return (%)]   (Proportion of fail cases (%))
   .Bars.I  :Bars   [Threshold return (%),Variable]   (value)

## Conclusions about the selection of threshold
I would choose threshold=3%, because with that we can expect that around 4% of the times the investment in VXX would have a negative return, meaning 96% of the times if we do a correct prediction of VIX going up, we can profit by investing in VXX. 